In [1]:
# Import needed libraries nad get needed dataset
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import pathlib
import numpy as np
from tensorflow import keras

_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir =  "/root/.keras/datasets/cats_and_dogs_filtered/train"
validation_dir = "/root/.keras/datasets/cats_and_dogs_filtered/validation"
train_dir = pathlib.Path(train_dir)
validation_dir = pathlib.Path(validation_dir)

In [2]:
# Gets classes names from files
CLASS_NAMES = np.array([item.name for item in train_dir.glob('*')])

In [3]:
# Augments data
@tf.function#(jit_compile=True) # If your system support XLA you can uncomment this 
def augment_data(image):
    image = tf.image.resize_with_crop_or_pad(image, 180, 180) # Add 6 pixels of padding
    image = tf.image.random_crop(image, size=[150, 150, 3]) # Random crop back to 28x28
    image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness

    return image

In [4]:
full_dataset = tf.data.Dataset.list_files("/root/.keras/datasets/cats_and_dogs_filtered/*/*/*")

In [5]:
DATASET_SIZE = len(full_dataset)
print("Dataset size: ", DATASET_SIZE)

Dataset size:  3000


In [6]:
# Determines train and validation dataset size
train_dataset = full_dataset.take(2000)
validation_dataset = full_dataset.skip(2000)

In [7]:
@tf.function#(jit_compile=True)
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  return parts[-2] == CLASS_NAMES

In [8]:
@tf.function#(jit_compile=True)
def load_img(image_path):
    img = tf.io.read_file(image_path)
    
    # https://stackoverflow.com/questions/44942729/tensorflowvalueerror-images-contains-no-shape
    img = tf.image.decode_image(img, 3, expand_animations=False)
    
    img = tf.cast(img, tf.float32)
    return img

In [9]:
# normalizing the images to [-1, 1]
@tf.function#(jit_compile=True)
def normalize(image):
    image = image / 255
    return image

In [10]:
@tf.function#(jit_compile=True)
def resize(image,height, width):
    image = tf.image.resize(image, (height, width),
                                 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return image

In [11]:
@tf.function#(jit_compile=True)
def load_image_with_label(image_path):
    label = get_label(image_path)
    img = load_img(image_path)
    return img, label

In [12]:
@tf.function#(jit_compile=True)
def load_image_train(image_file):
    image, label = load_image_with_label(image_file)
    image = augment_data(image)
    image = normalize(image)
    num_parallel_calls=tf.data.AUTOTUNE
    return image, label

In [13]:
@tf.function#(jit_compile=True)
def load_image_test(image_file):
    image, label = load_image_with_label(image_file)
    image = resize(image, 150, 150)
    image = normalize(image)
    num_parallel_calls=tf.data.AUTOTUNE
    return image, label

In [14]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 10

In [15]:
train_dataset = train_dataset.map(load_image_train)
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [16]:
validation_dataset = validation_dataset.map(load_image_test)
validation_dataset = validation_dataset.batch(BATCH_SIZE)
validation_dataset = validation_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [17]:
n_class = len(CLASS_NAMES)

In [18]:
# Turns model process data into mixed float16
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  Tesla K80, compute capability 3.7
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [19]:
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

Compute dtype: float16
Variable dtype: float32


In [20]:
# Model definition
from keras import layers
model = tf.keras.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(n_class, activation='sigmoid', dtype='float32'))

In [21]:
# Model compile
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [22]:
# Model fit
history = model.fit(train_dataset,
                    epochs=100,
                    validation_data=validation_dataset)

Epoch 1/100
63/63 [==============================] - 27s 312ms/step - loss: 0.6925 - acc: 0.5135 - val_loss: 0.7002 - val_acc: 0.5020
Epoch 2/100
63/63 [==============================] - 13s 205ms/step - loss: 0.6865 - acc: 0.5425 - val_loss: 0.6932 - val_acc: 0.5230
Epoch 3/100
63/63 [==============================] - 13s 205ms/step - loss: 0.6793 - acc: 0.5665 - val_loss: 0.6913 - val_acc: 0.5400
Epoch 4/100
63/63 [==============================] - 15s 232ms/step - loss: 0.6726 - acc: 0.5785 - val_loss: 0.7587 - val_acc: 0.4990
Epoch 5/100
63/63 [==============================] - 13s 207ms/step - loss: 0.6694 - acc: 0.6005 - val_loss: 0.6698 - val_acc: 0.5860
Epoch 6/100
63/63 [==============================] - 13s 206ms/step - loss: 0.6570 - acc: 0.6140 - val_loss: 0.7776 - val_acc: 0.5150
Epoch 7/100
63/63 [==============================] - 13s 205ms/step - loss: 0.6481 - acc: 0.6280 - val_loss: 0.6808 - val_acc: 0.5760
Epoch 8/100
63/63 [==============================] - 13s 205ms

KeyboardInterrupt: ignored

In [23]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: ignored